In [1]:
import itertools

In [2]:
# produces a list of q(x) with \sum n_P == 7 where Z(q(x)) = \sum n_P *P
# enumerate possibe tuples (n_P)_P
lst = []
for n_inf,n_0,n_1,n_2,n_3,n_4,n_5,n_6,n_7 in itertools.product(range(9),range(5),range(3),range(5),range(3),range(3),range(2),range(2),range(2)):
    if n_inf >= n_0 and n_0 >= n_1 and (n_inf + n_0 + n_1 + 2*n_2 + 3*n_3 + 4*n_4 + 5*n_5 + 6*n_6 + 7*n_7 == 7):
        lst.append((n_inf,n_0,n_1,n_2,n_3,n_4,n_5,n_6,n_7))

In [3]:
# produce a list of irreducible polynomials
R.<x> = PolynomialRing(GF(2))
irred_pol = {'0':[1]}
for d in range(8):
    for pol in R.polynomials(of_degree = d):
        if pol.is_irreducible() is True:
            try:
                irred_pol[str(d)].append(pol)
            except:
                irred_pol[str(d)] = [pol]

In [4]:
def pol_orbits(pol,stablist):
    pol = R(pol)
    if len(stablist) == 0:
        return {pol}
    elif len(stablist) == 1 and stablist[0] == x+1:
        return {pol,pol(stablist[0])}
    elif len(stablist) == 1 and stablist[0] == 1/x:
        return {pol,pol(stablist[0])*x^7}
    elif len(stablist) == 2:
        return {pol, pol(1/x)*x^7, pol(1/(x+1))*(x+1)^7, pol(x+1), pol(1 + 1/x)*x^7, pol(x/(x+1))*(x+1)^7}

In [5]:
# construct Q6 list from tuple (n_P)_P = (n_infty, n_0, ... , n_7)
Q6 = []

for ele in lst:
    
    # find factors of representatives
    possible_polys = {}
    for ind,count in enumerate(ele):
        if ind == 1 and count != 0:
            possible_polys['0'] = [x^count]
        elif ind == 2 and count != 0:
            possible_polys['1'] = [(x+1)^count]
        elif ind > 2 and count != 0:
            for pol_tup in itertools.product(irred_pol[str(ind-1)],repeat=int(count)):
                try:
                    possible_polys[str(ind-1)].append(prod(pol_tup))
                except:
                    possible_polys[str(ind-1)] = [prod(pol_tup)]
    polys = list(set([prod(i) for i in list(itertools.product(*possible_polys.values()))]))
    
    # find stabilisers of tuple (x+1 or 1/x)
    stab_list = []
    if ele[1] == ele[2]:
        stab_list.append(x+1)
    if ele[0] == ele[1]:
        stab_list.append(1/x)
    final = polys.copy()
    
    # check if polys are in the same equivalence class
    for pol in polys:
        tmp = pol_orbits(pol,stab_list)
        tmp2 = [orbs for orbs in tmp if orbs in polys]
        if len(tmp) == 1:
            continue
        elif len(tmp) > 1 and len(tmp2) > 1:
            ele = tmp2[0]
            for pols in tmp:
                try:
                    final.remove(pols)
                except:
                    continue
            final.append(ele)
        
    Q6 += final
Q6 = list(set(Q6))
print(len(Q6))

51


In [6]:
Q6 = [R(i) for i in Q6]

In [7]:
del lst, stab_list, polys, final, irred_pol

In [8]:
# Define PGL(2,F_2) action
G = GL(2,GF(2))
Glst = list(G.list())
def gaction(level,g,pol):
    g = g.matrix()
    return (g[1][0]*x + g[1][1])^(level)*pol((g[0][0]*x + g[0][1])/(g[1][0]*x + g[1][1]))

In [9]:
# same algorithm as genus >= 4
D14 = set(R.polynomials(max_degree=14))

In [10]:
%%time

from collections import defaultdict

Vq = defaultdict(list)
for q in Q6:
    print(q)
    Rq = R(q)
    
    #compute stabiliser G_q of q
    Gq = []
    if q == 1:
        Gq = [Glst[0],Glst[3]] # 1 is fixed under [1,0,0,1] and [1,1,0,1]
    else:
        for g in Glst:
            if gaction(7,g,Rq) == Rq:
                Gq.append(g)
        
    #find representative p of isomorphism class [y^2 + qy = p]
    D14copy = D14.copy()
    while D14copy:
        p = D14copy.pop()
        m = max(2*Rq.degree(),p.degree())
        if 13 <= m and m <= 14:
            Spprime = set([p + r^2 + Rq*r for r in R.polynomials(max_degree=7)])
            Sp = set()
            for pol in Spprime:
                Sp.update([gaction(14,g,pol) for g in Gq])
            D14copy = D14copy - Sp
            ele = Sp.pop()
            try:
                Vq[Rq].append(ele)
            except KeyError:
                Vq[Rq] = [ele]            
        else:
            continue

x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + 1
x^7 + x^6 + x^5 + x^4 + x^3 + x + 1
x^7 + x^6 + x^5 + x^4 + x^2 + x + 1
1
x^6 + x^5 + x^4 + x^3 + x^2 + x
x^6 + x^5 + x^4 + x^3 + x^2 + x + 1
x^4 + x^3
x^6 + x^5 + x^4 + x^3 + x
x^6 + x^5 + x^4 + x^2 + x
x^5 + x^4 + x^3 + x^2 + x
x^7 + x^6 + x^5 + x^3 + 1
x^7 + x^6 + x^5 + x^2 + 1
x^7 + x^6 + x^4 + x^3 + 1
x^6 + x^5 + x^4 + x^3 + 1
x^6 + x^5 + x^4 + x^2 + 1
x^6 + x^5 + x^3 + x^2 + 1
x^6 + x^5 + x^3 + x + 1
x^5 + x^4 + x^3 + x^2 + 1
x^5 + x^4 + x^3 + x + 1
x^6 + x^3 + x^2 + x + 1
x^5 + x^3 + x^2 + x
x^6 + x^5 + x
x^6 + x^4 + x
x^5 + x^4 + x
x^5 + x^3 + x
x^5 + x^2 + x
x^5 + x^4 + x^2
x^4 + x^3 + x
x^4 + x^2 + x
x^4 + x^3 + x^2
x^3 + x^2 + x
x^6 + x^5 + 1
x^6 + x^4 + 1
x^6 + x^3 + 1
x^6 + x
x^6 + x + 1
x^5 + x^2 + 1
x^5 + x + 1
x^5 + x^2
x^4 + x + 1
x^4 + x^3 + 1
x^4 + x^2 + 1
x^4 + x
x^4 + x^2
x^3 + x^2 + 1
x^3 + x^2
x^2 + x + 1
x^2 + x
x^3
x^2
x
CPU times: user 29min 32s, sys: 194 ms, total: 29min 32s
Wall time: 29min 32s


Check that the equation $y^2 + q(x)y = p(x)$ defines a nonsingular hyperelliptic curve:

For
- 0 $\not = q $ with $1 \leq deg(q) \leq g + 1$
- $p$ with $deg(p) \leq 2g + 2$
- $2g + 1 \leq max(2deg(q),deg(p)) \leq 2g + 2$

Then the above equation is nonsingular iff
- $gcd(q,p'^2 + q'^2p) = 1$
- either $deg(q) = g+1$ or $a_{2g+1}^2 \not = a_{2g+2}b_g^2$ where $p = \sum_{i=0}^{2g+2} a_i x^i, q = \sum_{i=0}^{g+1} b_ix^i$

In [11]:
%%time

#R1.<x> = PolynomialRing(QQ)
qphypell = {}

for q in Vq:
    plst = []
    for p in Vq[q]:
        if gcd(q, p.derivative()^2 + (q.derivative()^2)*p) == 1 and (R(q).degree() == 7 or mod(R(p).monomial_coefficient(x^13),2)^2 != mod((R(p).monomial_coefficient(x^14)),2)*mod(R(q).monomial_coefficient(x^6),2)^2):
            plst.append(R(p))
    qphypell[q] = plst

CPU times: user 2.21 s, sys: 2 ms, total: 2.21 s
Wall time: 2.21 s


In [12]:
del Vq, Q6, Gq, D14, D14copy

In [13]:
load("../../Shared/weil_poly_utils.sage")

In [14]:
d = defaultdict(list)
R1.<x,y> = PolynomialRing(GF(2))
for q in qphypell:
    for p in qphypell[q]:
        d[tuple(HyperellipticCurve(p,q).count_points(6))].append(y^2 + q*y + p)

In [15]:
sum(len(d[k]) for k in d)

4134

In [16]:
lst = list(d.keys())
for j in range(25):
    f = open('data/sorted_' + str(j) + '.txt','w+')
    for key in lst[134*j:134*j + 134]:
        for eqn in d[key]:
            f.write('[' + str(list(key)) + ',[' + str(eqn) + ']]\n')
    f.close()